In [ ]:
import sys
import os
########################################################################
from deepdreaming import img, display, deepdream as dd
from deepdreaming.utils import read_image_net_classes
from deepdreaming.config import DreamConfig

import numpy as np
from matplotlib import pyplot as plt

import torch
from torchvision.models import vgg16, VGG16_Weights

import cv2 as cv
########################################################################
SEED = 42
def set_seeds(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seeds(SEED)
########################################################################
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
TARGET_SHAPE = (224, 224, 3)


IMAGE_NET_CLASSES_PATH = "input/image_net_classes/imagenet_classes.txt"
INPUT_IMAGES_DIR = "input/deepdream-sample-images"
OUTPUT_IMAGES_DIR = "output"

# assert os.path.exists(IMAGE_NET_CLASSES_PATH), "Please provide a valid `IMAGE_NET_CLASSES_PATH`"
# assert os.path.exists(INPUT_IMAGES_DIR), "Please provide a valid `INPUT_IMAGES_DIR`"
# assert os.path.exists(OUTPUT_IMAGES_DIR), "Please provide a valid `DEEPDREAM_OUTPUT_PATH`"


device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).to(device).eval()
MODEL.requires_grad_(False);
IMAGE_NET_CLASSES = read_image_net_classes(IMAGE_NET_CLASSES_PATH)
print(MODEL)

In [ ]:
# deepdream = dd.DeepDream(MODEL, ["layer2[3].relu", "layer3[5].conv1"])
deepdream = dd.DeepDream(MODEL, ["features[13]", "features[22]", "features[25]", "features[27]"])
sample_images = [os.path.join(INPUT_IMAGES_DIR, file_name) for file_name in os.listdir(INPUT_IMAGES_DIR)]
sample_config = DreamConfig(
    pyramid_layers=3,
    pyramid_ratio=0.42,
    shift_size=32,
    num_iter=10,
    learning_rate=0.08
)

# nice configs:
# num_iter = 10
#   -> layers=9, ratio=4/5
#   -> layers=3, ratio=0.42

sample_images[0] = os.path.join(INPUT_IMAGES_DIR, "guys.jpg")
for sample_image in sample_images[::3]:
    # read 'n' dream
    image: np.ndarray = img.io.read_image(sample_image, TARGET_SHAPE)
    out = deepdream.dream(image, config=sample_config)

    # compare 'n' show
    display.display_two_img(image, out, figsize=(8, 8))
    plt.show()

    # transform for OpenCV
    out = img.proc.to_cv(out)

    # save
    file = f"{sample_image.split('/')[-1]}"
    fullpath = os.path.join(OUTPUT_IMAGES_DIR, file)
    success = cv.imwrite(fullpath, out)
    assert success, f"Image {fullpath} is not saved successfully..."
    break

## Using reference image
**Dreaming with the reference image provided**

In [ ]:
get_path = lambda prefix, name: os.path.join(prefix, name)

# some experimental layer with cool results - feel free to change!
deepdream = dd.DeepDream(MODEL, ["features[22]"])

# config for guided deepdream
guided_dream_config = DreamConfig(gradient_norm=True, pyramid_layers=4, num_iter=10)

### Dart Vader + Yoda

In [ ]:
image = img.io.read_image(get_path(INPUT_IMAGES_DIR, "yoda.jpeg"), TARGET_SHAPE)
reference = img.io.read_image(get_path(INPUT_IMAGES_DIR, "space.jpeg"), TARGET_SHAPE)

out = deepdream.dream(image, reference, config=guided_dream_config)
display.display_two_img(image, out, figsize=(8,8))